<a href="https://colab.research.google.com/github/eovallemagallanes/Digital-Image-Processing/blob/main/lecture12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wiener filter

$$
IOut(x) = \bar{x} + \frac{\sigma_L^2}{\sigma_L^2 + \sigma_G^2} (x - \bar{x}),
$$

where $\sigma_L^2$, and $\sigma_G^2$ are the local (patch) and global variance of the image, respectively. $\bar{x}$ is the mean intensity of the patch, and $x$ is the current intensity value at position $(i, j)$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import skimage
import skimage.color as skic
import skimage.filters as skif
import skimage.data as skid
import skimage.util as sku
from skimage.filters import median
from skimage.morphology import square

from scipy import signal
from sklearn.metrics import mean_squared_error
import time
from scipy.ndimage.filters import generic_filter


In [4]:
def wiener(img, D=0.1, kernel_size=3, central_pixel=True):
    # image shape
    h,w = img.shape

    # define kernel
    ks2= kernel_size//2
    N = kernel_size*kernel_size if central_pixel else (kernel_size*kernel_size)-1
    kernel = 1/N * np.ones((kernel_size, kernel_size))

    if not central_pixel:
        kernel[ks2, ks2] = 0

    # mean image
    img_mean = signal.convolve2d(img, kernel, mode='same')

    #get global variance
    varG = img.var()

    # get local variance
    img_var = generic_filter(img, np.var, size=kernel_size, mode='constant')

    # weiner filter
    img_filter = img_mean + ((img_var)/(img_var + varG)*(img - img_mean))

    # noise flag
    r = 1.*(abs(img - img_filter) > D)
    #plt.imshow(r)
    #plt.show()

    new_image = r*img_filter + (1-r)*img
    return new_image

In [5]:
skimage.util.random_noise

# working in range [0.0, 255.0]
img = skimage.img_as_float(skid.chelsea()) #*255.0

# convert to gray-scale image
gray_img = skic.rgb2gray(img)

# add s&p noise
#noise_img = sku.random_noise(gray_img, 'gaussian')
noise_img = sku.random_noise(gray_img, 's&p', salt_vs_pepper=0.5, amount=0.1)

In [6]:
start = time.time()
imgWiener = wiener(noise_img, D=0.2, kernel_size=3, central_pixel=False)
end = time.time()

print('time wiener filter: ', end - start, '(s)')

time wiener filter:  2.744729518890381 (s)


In [14]:
def runWeiner(img, img_noise, D=0.2, kernel_size=3, central_pixel=False):
    kernel_size = int(kernel_size)
    start = time.time()
    res_img = wiener(img_noise, D, kernel_size, central_pixel)
    end = time.time()
    print('Time Wiener filter: ', end - start, '(s)')

    mse = mean_squared_error(img, res_img)
    print('MSE: ', mse)

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 16))
    ax = axes.ravel()
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title("Original image")
    ax[1].imshow(img_noise, cmap='gray')
    ax[1].set_title("Noisy image")
    ax[2].imshow(res_img, cmap='gray')
    ax[2].set_title("Wiener filter")

    for a in ax:
        a.set_axis_off()

    plt.tight_layout()
    plt.show()

    return res_img

In [8]:
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import FloatSlider, Checkbox

In [15]:
central_widget = Checkbox(value=False, description='Central Pixel', disabled=False, indent=False)
threshold_widget = FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5, continuous_update=False)
kernel_widget = FloatSlider(min=3, max=11, step=2, value=3, continuous_update=False)

w = interactive(runWeiner,img=fixed(gray_img), img_noise=fixed(noise_img), D=threshold_widget, kernel_size=kernel_widget, central_pixel=central_widget);
display(w)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='D', max=1.0), FloatSlider(v…